In [31]:
from logging import exception
from conct_db import enginex
import pandas as pd
import numpy as np
import datetime
import urllib
from datetime import date



In [32]:
# from sqlalchemy.types import Integer
from sqlalchemy.types import DATE
# from sqlalchemy.types import VARCHAR 
# from sqlalchemy.types import INTEGER
# from sqlalchemy.types import Text
# from sqlalchemy.types import DATETIME

In [33]:
# parameters = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
#                                     "SERVER=WIN-SERVIDOR-BD\DWGRUPOTOTAL;"
#                                     "DATABASE=DW_Central;"
#                                     "UID=sa;"
#                                     "PWD=DWpln21")
periodo = datetime.datetime.now()

**UTILIZAR EL METODO CONNECT PARA EJECUTAR COSNULTAS**


Utilizo el text= para construir el query y ejecutarlo

In [34]:
# text=("""SELECT *
#         FROM BEES_Algoritmo
#         WHERE Periodo = '2022-07-01'""")
# with enginex.connect() as conn:
#     # conn.execute(text)
#     conn.exec_driver_sql(text)



**PARA BORRAR LOS DATOS DEL MES EN CURSO**

In [35]:
# text=("""DELETE BEES_Algoritmo
#     WHERE Periodo = '2022-07-01'""")
# enginex.execute(text)

**PARA LEER DESDE LA BASE DE DATOS**

In [36]:
# text=("""SELECT *
#         FROM BEES_Algoritmo
#         WHERE Periodo = '2022-07-01'""")
# 
# dataframendo= pd.DataFrame(enginex.execute(text))
# dataframendo.head(3)

In [37]:
df_original = pd.read_excel("y:/19-POWER BI/01-Total Bebidas/TACO GQ/Algoritmo_Julio.xlsx", sheet_name='Export', skipfooter=3)
df = pd.DataFrame(df_original)
df['POC_ID'] = df['POC_ID'].astype('string')
# df.dtypes
# df.sample(10)

In [38]:
# if df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)') == '704250':
#     df['COD_DISTRI'] = '70425'
# else:
#     df['COD_DISTRI'] = df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)')

In [39]:
df['COD_CLIENTE'] = df.POC_ID.str.extract('(0{2,}\d+)')
df['COD_DISTRI'] = df.POC_ID.str.extract('(\d{6,6}(?:\.\d+)?)')
df = df[df.COD_DISTRI.isin(['704250','328660','369117','369846','445625'])]
df['COD_DISTRI'] = df['COD_DISTRI'].astype('Int64')
df['COD_DISTRI'] = np.where(df['COD_DISTRI']==704250, 70425, df['COD_DISTRI'])
df['COD_CLIENTE'] = df['COD_CLIENTE'].astype('Int64')
df['Periodo'] = pd.to_datetime(date(periodo.year, periodo.month, 1))

# df.nunique()
# df.sample(5)

In [40]:
# df['Periodo'] = df[['Periodo']].astype(Date)
df.dtypes

POC_ID                             string
Ordenes Influenciadas               int64
Ordenes inf. Upsell                 int64
Ordenes inf.Quick Order             int64
Ordenes totales                     int64
COD_CLIENTE                         Int64
COD_DISTRI                         object
Periodo                    datetime64[ns]
dtype: object

In [41]:
# df['Periodo'] = pd.to_datetime('1/6/2022')
# df.convert_dtypes()
# df['POC_ID'].astype('Int64')
# df.dtypes

In [42]:
# conditionlist = [
#     (df['POC_ID']>=7042500000000) & (df['POC_ID']<7042600000000),
#     (df['POC_ID']>=32866000000000) & (df['POC_ID']<32866100000000),
#     (df['POC_ID']>=36911700000000) & (df['POC_ID']<36911800000000),
#     (df['POC_ID']>=36984600000000) & (df['POC_ID']<36984700000000),
#     (df['POC_ID']>=44562500000000) & (df['POC_ID']<44562600000000)]
# choicelist = ['70425', '328660', '369117', '369846', '445625']
# df['DISTRI'] = np.select(conditionlist, choicelist, default = 'OTRO DISTRI')
# df.sample

**BORRAMOS LAS COLUMNAS INECESARIAS Y REORDENAMOS EL DATAFRAME**

In [43]:
try: 
    df.drop(columns=['POC_ID'], inplace=True)
    df = df[['Periodo', 'COD_DISTRI', 'COD_CLIENTE', 'Ordenes inf.Quick Order', 'Ordenes inf. Upsell', 'Ordenes Influenciadas', 'Ordenes totales',]]
except: 
    print("Oh rallos y centellas, a ocurrido un error catastrofico! ")
# df.groupby(['DISTRI', 'Periodo']).sum()
# df.sample(10)

In [44]:
# df.drop(columns='DISTRI'=='OTRO DISTRI',axis=0)
# df
# df = df[df['DISTRI']!='OTRO DISTRI']
# df

In [45]:
def run():
    # try:
    #     engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(parameters))
    #     print("Conect succesfull")
    # except Exception:
    #     print("Errora in DataBase conection")
    try:    
        df.to_excel('y:/19-POWER BI/01-Total Bebidas/TACO GQ/Algoritmo_Julio_procesado.xlsx')
        df.to_sql('BEES_Algoritmo', con = enginex, if_exists= 'append', index=False, dtype={'Periodo': DATE})
        print("carga exitosa")
    except Exception:
        print("Error fatal irrecuperable")
text=("""DELETE BEES_Algoritmo
    WHERE Periodo = '2022-07-01'""")
enginex.execute(text)
run()

carga exitosa
